In [98]:
# load my working random forests model
# for top 5 features, plot distributions of each and probability of sucesss

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as math
from IPython.display import display
from IPython.core.pylabtools import getfigs
import sklearn.metrics
import seaborn as sns
import pickle

In [3]:
%matplotlib qt

In [4]:
saveas_path = '/Users/jamie/insight data/figures/'

In [5]:
outcomes_df = pd.read_pickle('outcomes_df.save')

In [100]:
success_is = np.where(outcomes_df['Outcome'] == 1)[0]
fail_is = np.where(outcomes_df['Outcome'] == 0)[0]
chance = len(success_is)/float(len(success_is) + len(fail_is))
print chance

0.419820997813


In [6]:
subset_predictor_df = pd.read_pickle('subset_predictor.save')
subset_predictor_df.head(3)

,Goal,Campaign_days,Title_n_words,Blurb_n_words,Month,Monday_day,Week_day,Hour,N_rewards,Min_reward,Median_reward,Max_reward,Reward_ship_days,Ships_intn,N_links,N_figs,Body_n_words,N_creator_backed
0,5000,30,2,17,8,12,6,20,8,10,175,5000,110,0,1,0,812,18
1,10000,30,8,21,6,30,5,14,10,5,125,5000,31,0,5,1,792,7
2,18000,27,5,14,2,25,1,8,19,5,500,10000,4,1,0,0,540,3


In [7]:
outcomes_df.head()

,Outcome
0,1
1,1
2,1
3,1
4,1


In [8]:
n_samples = np.shape(outcomes_df)[0]
print n_samples

133071


In [9]:
# now use training (80%) and test (20%) sets 
i_order = np.random.permutation(n_samples)
train_max_i = int(round(n_samples*.8))

train_is = i_order[:train_max_i]
test_is = i_order[train_max_i:]

In [10]:
X = subset_predictor_df
y = outcomes_df['Outcome'].values

In [11]:
# random forests model
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=400, max_depth=9)
clf.fit(X.ix[train_is,:],y[train_is])
y_1 = clf.predict(X.ix[test_is,:])

y_test = y[test_is]

In [12]:
print 'accuracy = ', round(clf.score(X.ix[test_is,:],y_test),4)

accuracy =  0.7352


In [13]:
sklearn.metrics.precision_recall_fscore_support(y_test,y_1)

(array([ 0.75723045,  0.70125155]),
 array([ 0.79633972,  0.65186501]),
 array([ 0.77629282,  0.67565702]),
 array([15354, 11260]))

In [ ]:
# learn what this means for each.

In [101]:
sklearn.metrics.roc_auc_score(y_test,y_1)

0.72410236245795201

In [14]:
feature_importances = clf.feature_importances_
print feature_importances

[ 0.25161571  0.03658046  0.00431227  0.00513067  0.0068378   0.00463458
  0.0024567   0.00567199  0.10709073  0.01790819  0.03171469  0.03726536
  0.0203552   0.0027049   0.05572952  0.01232661  0.06567288  0.33199174]


In [15]:
X.columns

Index([u'Goal', u'Campaign_days', u'Title_n_words', u'Blurb_n_words', u'Month',
       u'Monday_day', u'Week_day', u'Hour', u'N_rewards', u'Min_reward',
       u'Median_reward', u'Max_reward', u'Reward_ship_days', u'Ships_intn',
       u'N_links', u'N_figs', u'Body_n_words', u'N_creator_backed'],
      dtype='object')

In [16]:
feature_importance_is = np.argsort(feature_importances)

In [17]:
n_features = len(feature_importance_is)
print n_features

18


In [18]:
feature_importance_is_decending = feature_importance_is[-1::-1]

In [19]:
X.columns[feature_importance_is_decending]

Index([u'N_creator_backed', u'Goal', u'N_rewards', u'Body_n_words', u'N_links',
       u'Max_reward', u'Campaign_days', u'Median_reward', u'Reward_ship_days',
       u'Min_reward', u'N_figs', u'Month', u'Hour', u'Blurb_n_words',
       u'Monday_day', u'Title_n_words', u'Ships_intn', u'Week_day'],
      dtype='object')

In [20]:
sns.set_style("white")

In [21]:
plt.rcParams.update({'font.size': 26})

fig = plt.figure()
plt.title("Feature importances")
plt.bar(range(n_features), feature_importances[feature_importance_is_decending],
       color="b", align="center")
plt.xticks(range(n_features), X.columns[feature_importance_is_decending],rotation='vertical')
plt.xlim([-1, n_features])
plt.xlabel('Feature')
plt.ylabel('Feature importance (Gini importance)')
plt.show()

In [22]:
#saveas_path = '/Users/jamie/insight_data/figures/'
#plt.savefig(saveas_path + 'random forests importance june18.png',bbox_inches='tight',dpi=200) 

In [71]:
def plot_probability_funding_freq(subset_predictor_df,outcomes_df,feature_name,bins = range(12),step=1):
    import matplotlib.gridspec as gridspec
    fig = plt.figure()
    gs = gridspec.GridSpec(2,1)
    gs.update(wspace=0.1, hspace=0.1)
    
    
    hist_ax = plt.subplot(gs[1,0])  
    success_ax  = plt.subplot(gs[0,0],sharex=hist_ax) 
    
    this_predictor = subset_predictor_df[feature_name].values
    
    success_is = np.where(outcomes_df['Outcome'] == 1)[0]
    fail_is = np.where(outcomes_df['Outcome'] == 0)[0]
    chance = len(success_is)/float(len(success_is) + len(fail_is))
    
    hist_ax.hist(this_predictor[fail_is], bins=bins, normed=True, color="#F08080", alpha=.5);
    hist_ax.hist(this_predictor[success_is], bins=bins, normed=True, color="#6495ED", alpha=.5);
    
#     if step > 100:
#         hist_ax.set_xlim([-.5+-5*step,max(bins)+.5])
#     else:
#         hist_ax.set_xlim([-.5,max(bins)+.5])
        
    hist_ax.set_xlabel(feature_name)
    hist_ax.set_ylabel('Relative frequency')
    
    
    success_freq = np.empty_like(bins[:-1],dtype=float)

    for i,bin in enumerate(bins[:-1]):
        
        this_bin_is = np.intersect1d(np.where(subset_predictor_df[feature_name]>=bin)[0],
                                     np.where(subset_predictor_df[feature_name]<bin+step)[0])

        
        n_this_bin = len(this_bin_is)
        n_success = sum(outcomes_df.ix[this_bin_is,'Outcome'].values)
        if n_this_bin:
            success_freq[i] = float(n_success)/n_this_bin
        else:
            success_freq[i] = np.nan
    
        
    success_ax.axhline(chance,color='k',linewidth=.25)
    success_ax.plot(np.asarray(bins[:-1])+step/2.0,success_freq,'.',markersize=12)
    success_ax.set_ylim([0,1])
    success_ax.set_ylabel('Success probability')
    #success_ax.set_xticklabels([])
    
    saveas_path = '/Users/jamie/insight_data/figures/'
    plt.savefig(saveas_path + feature_name +' success rate and hist.png',bbox_inches='tight',dpi=200) 

In [72]:
X.columns[feature_importance_is_decending]

Index([u'N_creator_backed', u'Goal', u'N_rewards', u'Body_n_words', u'N_links',
       u'Max_reward', u'Campaign_days', u'Median_reward', u'Reward_ship_days',
       u'Min_reward', u'N_figs', u'Month', u'Hour', u'Blurb_n_words',
       u'Monday_day', u'Title_n_words', u'Ships_intn', u'Week_day'],
      dtype='object')

In [75]:
plot_probability_funding_freq(subset_predictor_df,outcomes_df,'N_creator_backed',range(0,12))

In [76]:
plot_probability_funding_freq(subset_predictor_df,outcomes_df,\
                              'Goal',range(0,30000,1000),1000)

In [77]:
plot_probability_funding_freq(subset_predictor_df,outcomes_df,'N_rewards',range(0,20))

In [78]:
plot_probability_funding_freq(subset_predictor_df,outcomes_df,'Body_n_words',range(0,2000,50),100)

In [79]:
plot_probability_funding_freq(subset_predictor_df,outcomes_df,'N_links',range(0,12))

In [80]:
plt.close('all')

In [82]:
plot_probability_funding_freq(subset_predictor_df,outcomes_df,'Max_reward',range(0,1000,100),100)

In [96]:
plot_probability_funding_freq(subset_predictor_df,outcomes_df,'Campaign_days',range(0,61,1))

In [86]:
plot_probability_funding_freq(subset_predictor_df,outcomes_df,'Reward_ship_days',range(0,365,7))

In [90]:
plot_probability_funding_freq(subset_predictor_df,outcomes_df,'Ships_intn',range(-1,3,1))

In [92]:
plot_probability_funding_freq(subset_predictor_df,outcomes_df,'Monday_day',range(1,32,1))

In [95]:
plot_probability_funding_freq(subset_predictor_df,outcomes_df,'Week_day',range(0,8,1))

In [97]:
print chance

NameError: name 'chance' is not defined